In [1]:
import json
from web3 import Web3
import pandas as pd
%precision 30

'%.30f'

In [7]:
with open('/mnt/c/blockchain/api_keys/metamask.json') as f:
        maskdata = json.load(f)
pubkey = maskdata['account1']['public']
privkey = maskdata['account1']['private']

pubkey2 = '0x83b8E4C91Cd770F2ef10105A5c00BC7b94279087'

with open("./contract_address.json") as f:
    contractaddy = json.load(f)

with open("./ABI.json") as f:
    ABI = json.load(f)

In [71]:
class web3kit:
    def connect(self, node='infura', network='mainnet', connection='HTTP'):
        """
        Node: geth, infura
        Network: mainnet, ropsten, rinkeby...
        Connection: IPC, Websocket, HTTP
        """
        
        if connection == 'IPC':
            if node == 'infura':
                    raise ValueError('Infure not available with connection type IPC')
            else:
                if network == 'mainnet':
                        self.web3 = Web3(Web3.IPCProvider('~/.ethereum/geth.ipc'))
                else:
                    self.web3 = Web3(Web3.IPCProvider('~/.ethereum/geth' + network + '/geth.ipc'))

        elif connection == 'Websocket':
            if node == 'infura':
                with open('/mnt/c/blockchain/api_keys/infura.json') as f:
                    self.infdata = json.load(f)
                self.infurl = "wss://" + network + ".infura.io/ws/v3/" + self.infdata['web3_eth1']['project_id']
                self.web3 = Web3(Web3.WebsocketProvider(self.infurl))
            else:
                raise ValueError('Geth websocket not yet implemented')
        
        elif connection == 'HTTP':
            if node == 'infura':
                with open('/mnt/c/blockchain/api_keys/infura.json') as f:
                    self.infdata = json.load(f)
                self.infurl = "https://" + network + ".infura.io/v3/" + self.infdata['web3_eth1']['project_id']
                self.web3 = Web3(Web3.HTTPProvider(self.infurl))
            else:
                raise ValueError('Geth HTTP not yet implemented')
        else:
            raise ValueError('Invalid connection type')
            
        return self.web3


    def get_balance(self, token, public_address, contract_address_json, ABI, currency_quote):
        """
        Get the balance of a particular token
        """
        if token == 'ether' or 'ETH':
            self.wei = web3.eth.getBalance(public_address)

        else:
            self.cpubkey = Web3.toChecksumAddress(public_address)
            
            try:
                self.contract_address_json = contract_address_json[token]
            except Exception:
                raise ValueError('Coin ABI not in ABI json')
            self.ccontractaddy = Web3.toChecksumAddress(self.contract_address_json)
            self.contract = web3.eth.contract(self.ccontractaddy, abi=ABI)
            self.wei = self.contract.functions.balanceOf(self.cpubkey).call()
        self.balance = float(web3.fromWei(self.wei, currency_quote))
        return self.balance

    # Gas Fee Estimation
    def gas_limiter(self, transaction_speed='average', custom_maxpriorityfee=None):
        """
        Estimates priority fee and gets current base fee
        to determine a reasonable maxFeePerGas for the
        transaction.  Allows for a user specified
        transaction_speed, which applied to the
        maxpriorityfee to either increase/decrease
        the tip to miners, speeding/slowing the
        time to process the transaction.

        transaction_speed: 'slow', 'average', 'fast', 'custom_multiple'

        slow = .75
        average = 1
        fast = 1.25

        maxpriorityfee_est * transaction_speed = maxpriorityfee

        If a custom_priorityfee is specified it will override
        the default maxpriorityfee with the specified amount.
        Must be entered in Wei.
        """
        # Pulls an estimate of current priority fees (miner fees) using Geth's calculation (look online for current info on how this is done)
        if custom_maxpriorityfee:
            self.maxpriorityfee = custom_maxpriorityfee
        else:
            if type(transaction_speed) == int or float:
                self.multiple = transaction_speed
            else:
                self.transaction_speed = transaction_speed.lower()
                self.multiple_dict = {'slow': .75, 'average': 1, 'fast': 1.25}
                if transaction_speed in self.multiple_dict:
                    self.multiple = self.multiple_dict[transaction_speed]
                else:
                    raise ValueError('transaction_speed specified is not a recognized value')

            # Pulls the current base fee from the next block after the pending one, the next block's base fee is predermined by the pending block and is therefore certain
            self.maxpriorityfee_est = web3.eth.max_priority_fee
            self.maxpriorityfee = self.maxpriorityfee_est * self.multiple

        self.currentbasefee = web3.toWei(web3.eth.fee_history(1, 'pending')['baseFeePerGas'][-1], 'gwei')
        # This is the agreed seemingly universally agreed upon formula for the maxFeePerGas
        self.maxgasfee = (2 * self.currentbasefee) + self.maxpriorityfee
        return (self.maxgasfee, self.maxpriorityfee)


In [3]:
node = 'infura'  # geth, infura
network = 'ropsten' # mainnet, ropsten, rinkeby, polygon-mainnet
connection = 'HTTP' # HTTP, IPC, Websocket
token = 'ether' # ETH, USDT, USDC, etc...
currency_quote = 'ether'

web3 = web3kit().connect(node, network, connection)
print(web3.isConnected())


True


In [8]:
balance

5.198583868997070922546299698297

In [32]:
# Creating and sending transaction
maxgas = web3kit().gas_limiter(transaction_speed='average')

nonce = web3.eth.getTransactionCount(pubkey)
tx = {
    'nonce': nonce,
    'to': pubkey2,
    'value': web3.toWei(.1, 'ether'),
    'maxFeePerGas': maxgas[0],
    'maxPriorityFeePerGas': maxgas[1]
}

signed_tx = web3.eth.account.signTransaction(tx, privkey)
tx_hash = web3.eth.sendRawTransaction(signed_tx.rawTransaction)

In [9]:
# Getting post transaction info

receipt = web3.eth.wait_for_transaction_receipt(tx_hash, timeout=420)
transaction_data = web3.eth.get_transaction(tx_hash)

final_gas = web3.fromWei(receipt['gasUsed'], 'ether')

value = tx['value']

actual_cost = value + final_gas

NameError: name 'tx_hash' is not defined